# Teil 3, Demo 1: Datenabfrage & -visualisierung aus Open Data-Portal Mannheim

Ziel dieser Demo ist das Laden und Visualisieren von Daten eines Open Data-Portals, hier das Portal der Stadt Mannheim.

## Bibliotheken & Konfiguration

In diesem Abschnitt werden benötigte Programmpakete geladen und Konfigurationsvariablen z.B. für die Datenquellen gesetzt.

Hinweis: auf Google Colab kann das Laden der Pakete vor allem beim ersten Aufruf einige Minuten dauern. Bitte führen Sie diesen Block dann nicht erneut aus, sondern warten die Ausführung ab.

In [ ]:
packages <- c("readr", "tidyr", "dplyr", "ggplot2")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

## Bevölkerungsbestand vom Portal einlesen

In diesem Abschnitt laden wir Bevölkerungsdaten mit dem `read_delim()`-Befehl aus dem Open Data-Portal der Stadt Mannheim:
* in der ersten Zeile geben wir die Adresse des herunterzuladenden Datensatzes an (Caveat: die Adresse kann sich von Jahr zu Jahr ändern, da die Stadt den Datensätz jährlich erweitert. Falls es eine Fehlermeldung gibt, können Sie die Jahreszahl "2023" gegen das aktuelle Vorjahr austauschen, dann sollte es wieder funktionieren.)
* in der zweiten Zeile geben wir sprechende Spaltennamen für den Datensatz an, da die Bezeichner aus dem Open Data-Portal eher kryptisch sind
* in der dritten Zeile geben wir explizite Datentypen an (i=Integer-Zahl, c=Text, d=Fließkommazahl "double")
* in der letzten Zeile geben wir an, dass die erste Zeile übersprungen werden soll, da sie die (kryptischen) Spaltennamen des Portals enthält, außerdem legen wir das Trennzeichen zwischen den Spalten als Semikolon fest

Mit dem `head()`-Befehl zeigen wir die ersten 6 Zeilen des Datensatzes als Tabelle an.

In [ ]:
mannheim.population.raw <- read_delim("https://mannheim.opendatasoft.com/explore/dataset/bevolkerungsbestand-in-mannheim-2009-2023/download/?format=csv&timezone=Europe/Berlin&lang=de&use_labels_for_header=true&csv_separator=%3B",
                                      col_names =    c("id", "jahr", "stadtteil_id", "stadtteil_name", "n_gesamt", "n_weiblich", "n_ohne_migration", "n_mit_migration", "n_auslaender"),
                                      col_types = cols("i",  "i",    "c",            "c",              "d",        "d",          "d",                "d",               "d"),
                                      skip = 1, delim = ";")
head(mannheim.population.raw)

## Tabelle aus "Querformat" in "Längsformat" kippen

Mit dem Befehl `pivot_longer()` klappen wir den geladenen Datensatz aus einem "Querformat" in ein "Längsformat" um. Kennzahlen, die dabei zunächst in Spalten mit explizitem Namen "nebeneinander" standen, finden wir danach "untereinander", wobei alle Kennzahlen in einer gemeinsamen Spalte stehen und die Bedeutung in einer zusätzlichen Spalte (`variable`) angegeben wird. Dieses Format hat den Vorteil, dass verschiedene Kennzahlen leichter gemeinsam verarbeitet und dargestellt werden können. Man spricht hier auch von einem "generischen" oder auch "entity-attribute-value"-Format.

In [ ]:
mannheim.population.long <- mannheim.population.raw %>% pivot_longer(!c(id, jahr, stadtteil_id, stadtteil_name), names_to="variable", values_to="n")
head(mannheim.population.long)

## Visualisierung Gesamtbevölkerung vs. Zeit

In diesem Block erstellen wir ein einfaches Säulendiagramm, das die Gesamt-Bevölkerungsentwicklung der Stadt Mannheim über die Zeit darstellt:
* der `options()`-Befehl in der ersten Zeile legt die Größe der Grafik im Notebook fest
* der `ggplot()`-Befehl erzeugt eine Grafik, für die nacheinander verschiedene Angaben gemacht werden:
  * im `data`-Attribut legen wir den Quelldatensatz fest: wir nutzen dazu die pivotierte Version unseres Bevölkerungsdatensatzes und filtern ihn auf die Variable `n_gesamt` (=Gesamtbevölkerung) und gruppieren & summieren anschließend die Daten der einzelnen Stadtteile pro Jahr auf
  * mit der `aes`-Funktion (aesthetics) legen wir die Bedeutung der Achsen fest: hier X-Achse für das Jahr und y-Achse für die Bevölkerungszahl
  * mit der `geom_bar()`-Funktion legen wir das Säulendiagramm als Darstellungsform fest

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = mannheim.population.long %>% filter(variable == "n_gesamt") %>% group_by(jahr) %>% summarize(n = sum(n), .groups="keep"),
       aes(x = jahr, y = n)) +
    geom_bar(stat = "identity")

## Visualisierung Migrationsanteil vs. Zeit

Neben der Gesamtbevölkerung enthält der Datensatz auch die Zahl der Einwohner:innen mit Migrationshintergrund. Da diese Variable sich über die Zeit verändert, nutzen wir sie hier zur Darstellung weiterer Visualisierungsformen.

Im folgenden Block wird der Migrationsanteil über die Zeit als Anteil der Gesamtbevölkerung angezeigt:
* wir wählen diesmal nicht die Gesamtbevölkerung, sondern die beiden Variablen `n_mit_migration` und `n_ohne_migration` aus und gruppieren/summieren anschließend wieder die Stadtteildaten nach Jahr
* in der `aes()`-Funktion geben wir zusätzlich an, dass der Name der Kennzahl (`variable`) zur Färbung der Balken genutzt werden soll
* durch das zusätzliche Attribut `position = "fill"` geben wir an, dass nicht die absoluten Zahlen dargestellt werden sollen, sondern der Anteil in Bezug auf die Gesamtzahl pro Jahr

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = mannheim.population.long %>% filter(variable %in% c("n_mit_migration", "n_ohne_migration")) %>% group_by(jahr, variable) %>% summarize(n = sum(n), .groups="keep"),
       aes(x = jahr, y = n, fill = variable)) +
    geom_bar(stat = "identity",
             position = "fill")

## Visualisierung Migrationsanteil vs Stadtteil

Im folgenden Block ändern wir die X-Achse und stellen die Daten nicht mehr nach Jahr, sondern nach Stadtteil dar. Außerdem passen wir die Achsenbeschrift so an, dass die gegenüber den Jahren längeren Stadtteilnahmen hochkant geschrieben werden und so alle nebeneinander in die Grafik passen.
* in der `aes()`-Funktion setzen wir für die X-Achse die Spalte `stadtteil_name` statt `jahr` ein
* wir fügen die `theme()`-Funktion hinzu, in der wir den Winkel der X-Achsen-Beschriftung auf 90° setzen

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = mannheim.population.long %>% filter(variable %in% c("n_mit_migration", "n_ohne_migration")) %>% group_by(stadtteil_name, variable) %>% summarize(n = sum(n), .groups="keep"),
       aes(x = stadtteil_name, y = n, fill = variable)) +
    geom_bar(stat = "identity",
             position = "fill") +
    theme(axis.text.x = element_text(angle = 90))

## Visualisierung Migrationsanteil vs. Stadteil & Zeit

In diesem Block wollen wir sowohl die Stadtteile als auch die Entwicklung über die Zeit gemeinsam darstellen. Hierzu verwenden wir die `facet_wrap()`-Funktion, bei der wir eine Spalte angeben können, nach der die Grafik in kleinere Teile aufgeteilt werden soll.
* bei der Vorbereitung der Daten müssen wir die Aggregation anpassen, da wir bisher ja nur nach Variable und Jahr summiert hatten, so dass die Stadtteildaten in der Aggregation verschwinden; durch Aufnahme der Spalte `stadtteil_name` in der `group_by()`-Funktion behalten wir die Stadtteile bei
* in der `facet_wrap`-Funktion geben wir die so erhaltene Spalte an, und ggplot erzeugt automatisch eine Matrix von Teil-Diagrammen für die einzelnen Stadtteile.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)
ggplot(data = mannheim.population.long %>% filter(variable %in% c("n_mit_migration", "n_ohne_migration")) %>% group_by(jahr, stadtteil_name, variable) %>% summarize(n = sum(n), .groups="keep"),
       aes(x=jahr, y=n, fill=variable)) +
    geom_bar(stat = "identity",
             position = "fill") +
    theme(axis.text.x = element_text(angle = 90)) +
    facet_wrap(~stadtteil_name)

Das schöne an der `facet_wrap()`-Funktion ist, dass ggplot automatisch dafür sorgt, dass für alle Teildiagramme die gleichen Achsen verwendet werden, so dass die Zahlen vergleichbar sind. Um das zu verdeutlichen, stellen wir im folgenden Diagramm von der Darstellung der Verteilung auf absolute Zahlen um. Der Migrationsanteil kann so auch im Verhältnis der Gesamtbevölkerungszahl der einzelnen Stadtteile, aber auch der Stadtteile im Vergleich beurteilt werden.
* für die Umstellung auf absolute Zahlen müssen wir nur in der `geom_bar()`-Funktion das Attribut `position = "fill"` entfernen

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)
ggplot(data = mannheim.population.long %>% filter(variable %in% c("n_mit_migration", "n_ohne_migration")) %>% group_by(jahr, stadtteil_name, variable) %>% summarize(n = sum(n), .groups="keep"),
       aes(x=jahr, y=n, fill=variable)) +
    geom_bar(stat = "identity") +
    theme(axis.text.x = element_text(angle = 90)) +
    facet_wrap(~stadtteil_name)